In [1]:
import pandas as pd

import json

Откроем все файлы со сгенерированными дистракторами:

In [ ]:
chatgpt4_result = pd.read_csv(
    "../data_output_table/predict_distractors/Output_ChatGPT4o.csv", sep = ';', index_col="Unnamed: 0"
)

In [3]:
true_distractors = [
    [item.strip() for item in line.split('\n')]
    for line in chatgpt4_result["distractors"]
]

In [ ]:
fp_dict = {
    "BartDG": "../data_input/Ru-RC-DG/BartDGOutputEGE.xlsx",
    "BartDG_PM": "../data_input/Ru-RC-DG/BartDG_PM_OutputEGE.xlsx",
    "BartDG_ANPM": "../data_input/Ru-RC-DG/BartDG_ANPM_OutputEGE.xlsx",
    "MuSeRC_GPT3": "../data_input/Ru-RC-DG/metrics_muserc_ege_gpt3.csv",
    "MuSeRC_T5": "../data_input/Ru-RC-DG/metrics_muserc_ege_t5.csv",
    "RuRace_GPT3": "../data_input/Ru-RC-DG/MetricsEGERuGPT3.xlsx",
    "RuRace_T5": "../data_input/Ru-RC-DG/MetricsEGET5.xlsx",
    "Deepseek": "../data_output_table/predict_distractors/Output_DeepSeek.csv",
    "ChatGPT4o": "../data_output_table/predict_distractors/Output_ChatGPT4o.csv"
}

In [ ]:
def read_result_file(file: str) -> list[list[str]]:
    if file.endswith(".xlsx"):
        data = pd.read_excel(file, index_col="Unnamed: 0")
    elif file.endswith(".csv"):
        data = pd.read_csv(file, index_col="Unnamed: 0", sep=';')
    else:
        raise ValueError(
            f"Only csv and xlsx files are supported, got {file}"
        )

    if ("t5" in file.lower()) or ("bart" in file.lower()):
        output = [
            [item.strip('" ') for item in value.split(';')]
            for value in data["output"].to_list()
        ]
    elif (
        "gpt3" in file.lower()
    ) or (
        "deepseek" in file.lower()
    ) or (
        "chatgpt" in file.lower()
    ):
        output = [
            [item.strip('" ') for item in line.split("\n")]
            for line in data["output"].to_list()
        ]

    return output

In [ ]:
data_dict = {
    key: read_result_file(val) for key, val in fp_dict.items()
}
data_dict["true_distractors"] = true_distractors

In [8]:
def remove_duplicates(l: list) -> list:
    outp = []

    for item in l:
        if item not in outp:
            outp.append(item)

    return outp

In [9]:
for key in data_dict:
    for index, item in enumerate(data_dict[key]):
        data_dict[key][index] = remove_duplicates(item)

In [ ]:
with open ("../data_input/data_dict.json", 'w', encoding="utf8") as outp:
    json.dump(data_dict, outp, ensure_ascii=False, indent=4)